In [2]:
from __init__ import *

In [4]:
import natasha

In [11]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance
geolocator = Nominatim(user_agent="my test app")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [3]:
with sqlite3.connect('data/avito_real_estate.db') as db:
    df = pd.read_sql('select * from moskva_RE',con=db)

In [5]:
doc = natasha.Doc(df['text'].str.extract('([А-я]+ ремонт)').fillna('').apply('|'.join)[0])

In [6]:
segmenter = natasha.Segmenter()
morph_vocab = natasha.MorphVocab()
emb = natasha.NewsEmbedding()
morph_tagger = natasha.NewsMorphTagger(emb)

In [7]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [8]:
for token in doc.tokens:
     token.lemmatize(morph_vocab)

In [9]:
df['price'] = df['price'].astype(int)

In [10]:
df['rooms'] = df['title'].str.split(', ',expand=True)[0].str.extract('(\d)').astype(int)
df['m2']=df['title'].str.split(', ',expand=True)[1].str.replace(',','.').str.extract('(\d.*\d)').astype(float)
df[['floor','max_floor']]=df['title'].str.extract('(\d+/\d+)')[0].str.split('/',expand=True)
df['has_furniture']=df['text'].str.contains('мебель')
df['n_words'] = df['text'].str.count(' ')
df['repair_type'] = ' '.join([token.lemma for token in doc.tokens if token.lemma != 'ремонт']).split('|')
df['metro_name']=df['metro'].str.split(', ',expand=True)[0]
df['metro_dist']=df['metro'].str.split(', ',expand=True)[1].str.replace(',','.').str.extract('(\d+.\d?)').astype('float')[0].apply(lambda x: x*1000 if x<10 else x)
df['rubm2'] = (df['price'] / df['m2']).round()
df['room_size']= (df['m2'] / df['rooms']).round()
df['street']=df['adress'].str.split(', (\d.*)',expand=True)[0]
df['build']=df['adress'].str.split(', (\d.*)',expand=True)[1]

In [12]:
mid = geolocator.geocode("Moscow")

In [13]:
dist_ = {}
for i in tqdm(df['street'].unique()):
    try:
        dist_[i]=distance.distance(geocode(f'Москва {i}').point,mid.point).km
    except:
        print(i)

  1%|          | 7/841 [00:06<13:15,  1.05it/s]

Волгоградский пр-т


  2%|▏         | 17/841 [00:16<13:48,  1.01s/it]

пр-т Маршала Жукова


  6%|▌         | 47/841 [00:46<13:17,  1.00s/it]

...

 98%|█████████▊| 828/841 [13:48<00:12,  1.00it/s]

6-я Радиальная ул., вл7к25


100%|█████████▉| 840/841 [14:00<00:01,  1.00s/it]

пос. Битца, микрорайон Южная Битца, Южный б-р


100%|██████████| 841/841 [14:01<00:00,  1.00s/it]


In [16]:
df['dfc']=df['street'].map(dist_)

In [26]:
with sqlite3.connect('data/avito_real_estate.db') as db:
    df.drop('index',axis=1).to_sql('moscow_features',con=db,if_exists='append')